In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

# Upload file
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)

# Filter
df = df.dropna(subset=['Player', 'Minutes Played', 'Age', 'Touches'])
df['Minutes Played'] = pd.to_numeric(df['Minutes Played'], errors='coerce')
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['Touches'] = pd.to_numeric(df['Touches'], errors='coerce')
df = df[(df['Minutes Played'] >= 800) & (df['Age'] < 24) & (df['Touches'] > 0)].copy()

# Normalize
per_touch_metrics = [
    'Assists', 'Key Passes', 'Progressive Passes',
    'Progressive Carries', 'Carries into the Penalty Area',
    'Shot-Creating Actions','Goal-creating Actions', 'Expected Assisted Goals',
    'Progressive Passing Distance', 'Progressive Carrying Distance', 'Passes into the Penalty Area'
]

for col in per_touch_metrics:
    if col in df.columns:
        df[col + ' per 100 touches'] = df[col] / df['Touches'] * 100

per_touch_cols = [col + ' per 100 touches' for col in per_touch_metrics if col + ' per 100 touches' in df.columns]
df = df.dropna(subset=per_touch_cols)

# Position filter
valid_positions = ['MF', 'MF,FW', 'FW,MF', 'FW']
df = df[df['Position'].isin(valid_positions)].copy()

def role_mapping(pos):
    if pos == 'MF': return 'MF'
    if pos in ['MF,FW', 'FW,MF']: return 'MFFW/FWMF'
    if pos == 'FW': return 'FW'
    return 'Other'

df['RefinedRole'] = df['Position'].apply(role_mapping)
df = df[df['RefinedRole'] != 'Other']

# weighted scoring
role_weight_trees = {
      "MF": {
        "Possession": (0.35, {
            "Progressive Carries per 100 touches": 0.15,
            "Progressive Passes per 100 touches": 0.15,
            "Progressive Passing Distance per 100 touches": 0.15,
            "Progressive Carrying Distance per 100 touches": 0.15,
            "Carries into the Penalty Area per 100 touches": 0.2,
            "Passes into the Penalty Area per 100 touches": 0.2
        }),
        "Chance Creation": (0.65, {
            "Expected Assisted Goals per 100 touches": 0.25,
            "Assists per 100 touches": 0.2,
            "Shot-Creating Actions per 100 touches": 0.25,
            "Goal-creating Actions per 100 touches": 0.2,
            "Key Passes per 100 touches": 0.1
        })
    },
      "MFFW/FWMF": {
         "Possession": (0.35, {
            "Progressive Carries per 100 touches": 0.15,
            "Progressive Passes per 100 touches": 0.15,
            "Progressive Passing Distance per 100 touches": 0.15,
            "Progressive Carrying Distance per 100 touches": 0.15,
            "Carries into the Penalty Area per 100 touches": 0.2,
            "Passes into the Penalty Area per 100 touches": 0.2
        }),
        "Chance Creation": (0.65, {
            "Expected Assisted Goals per 100 touches": 0.25,
            "Assists per 100 touches": 0.2,
            "Shot-Creating Actions per 100 touches": 0.25,
            "Goal-creating Actions per 100 touches": 0.2,
            "Key Passes per 100 touches": 0.1
        })
    },
      "FW": {
         "Possession": (0.35, {
            "Progressive Carries per 100 touches": 0.15,
            "Progressive Passes per 100 touches": 0.15,
            "Progressive Passing Distance per 100 touches": 0.15,
            "Progressive Carrying Distance per 100 touches": 0.15,
            "Carries into the Penalty Area per 100 touches": 0.2,
            "Passes into the Penalty Area per 100 touches": 0.2
        }),
        "Chance Creation": (0.65, {
            "Expected Assisted Goals per 100 touches": 0.25,
            "Assists per 100 touches": 0.2,
            "Shot-Creating Actions per 100 touches": 0.25,
            "Goal-creating Actions per 100 touches": 0.2,
            "Key Passes per 100 touches": 0.1
        })
    }
}

results = []

for role, tree in role_weight_trees.items():
    print(f"\n🔍 Processing role: {role}")

    subset = df[df['RefinedRole'] == role].copy()
    if subset.empty:
        print(f"⚠️ No players found for role: {role}")
        continue

    all_metrics = [metric for _, sub in tree.values() for metric in sub]
    missing = [m for m in all_metrics if m not in subset.columns]
    if missing:
        print(f"⚠️ Skipping {role}: missing metrics -> {missing}")
        continue

    X = subset[all_metrics].copy().dropna()
    if X.empty:
        print(f"⚠️ No complete data rows for role: {role}")
        continue

    subset = subset.loc[X.index]
    scaler = StandardScaler()
    Z = scaler.fit_transform(X)
    Z = pd.DataFrame(Z, columns=X.columns, index=subset.index)

    category_scores = {}
    for cat, (cat_weight, submetrics) in tree.items():
        sub_df = Z[list(submetrics.keys())]
        sub_weights = list(submetrics.values())
        category_score = sub_df.mul(sub_weights, axis=1).sum(axis=1)
        category_scores[cat] = category_score * cat_weight

    final_score = sum(category_scores.values())
    subset['Score'] = final_score

    # Top 40 players
    top_players = subset.sort_values('Score', ascending=False).head(40)
    results.append((role, top_players[['Player', 'Age', 'Score']]))

    # Plot
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Score', y='Player', data=top_players, palette='viridis')
    plt.title(f"Top 40 Players - {role}")
    plt.xlabel("Score")
    plt.ylabel("Player")
    plt.tight_layout()
    plt.show()